In [ ]:
from datasets import load_dataset

ds = load_dataset("matejklemen/clc_fce")

Using the latest cached version of the dataset since matejklemen/clc_fce couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/frankli/.cache/huggingface/datasets/matejklemen___clc_fce/default/2.1.0/e4b0c1de4dc1e9c9ee9370c603c7010dcd255e3d (last modified on Thu Dec 12 17:00:08 2024).


In [28]:
all_ds= [ds["train"], ds["test"], ds["validation"]]
print(all_ds[0].column_names)

['src_tokens', 'tgt_tokens', 'corrections']


In [35]:
import re

import re

def detokenize(tokens):
    sentence = ""
    open_double_quote = True
    open_single_quote = True
    
    for token in tokens:
        if token == '"':
            if open_double_quote:
                sentence += ' "' if sentence and not sentence[-1].isspace() else '"'
                open_double_quote = False
            else:
                sentence = sentence.rstrip() + '" '
                open_double_quote = True
        elif token == "'":
            if open_single_quote:
                sentence += " '" if sentence and not sentence[-1].isspace() else "'"
                open_single_quote = False
            else:
                sentence = sentence.rstrip() + "' "
                open_single_quote = True
        elif token in {".", ",", "!", "?", ":", ";", "-"}:
            if sentence and sentence[-1] == ' ':
                sentence = sentence[:-1]
            sentence += token
        elif token in {")", "]", "}"}:
            sentence = sentence.rstrip() + token
        elif token in {"(", "[", "{"}:
            sentence += token
        else:
            no_space_tokens = ["'s", "n't", "'S", "'m", "'ll", "'ve", "'d", "'re", "--", ".-"]
            prev_no_space_tokens = ["'", "$", "-", '"', '"', ' ', '£']
            if sentence and sentence[-1] not in prev_no_space_tokens and token not in no_space_tokens:
                sentence += " "
            sentence += token
    return sentence

# Example usage
tokens = [
    "I", "am", "writing", "in", "order", "to", "express", "my", 
    "disappointment", "about", "your", "musical", "show", "\"", 
    "Over", "the", "Rainbow", "\"", "."
]
sentence = detokenize(tokens)
print(sentence)

I am writing in order to express my disappointment about your musical show "Over the Rainbow".


In [37]:
import pandas as pd
before_statements = []
after_statements = []
corrections = []
tokens = set()
for ds in all_ds:
    for example in ds:
        before = example['src_tokens']
        after = example['tgt_tokens']
        correct = example['corrections']
        # for token in before:
        #     if len(token) < 4 and token not in tokens and ("'" in token or "." in token or "-" in token or "-" in token or "*" in token or '£' in token):
        #         tokens.add(token)
        #         print(token)
        #         print(before)
        #         print(detokenize(before))
        before_statements.append(detokenize(before))
        after_statements.append(detokenize(after))
        corrections.append(correct)
tokens = list(tokens)
for token in tokens:
    print(token)

In [38]:
data = {
    "Before Statements": before_statements,
    "After Statements": after_statements,
    "Error Correction": corrections
}
df = pd.DataFrame(data)
df.to_csv("CLC_FCE.csv", index = False)

In [1]:
from datasets import load_dataset
dataset = load_dataset("MohamedAshraf701/lang-8")

/Users/frankli/miniconda3/envs/final_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 610053/610053 [00:00<00:00, 3397128.47 examples/s]


In [ ]:
ds = dataset['train']

In [4]:
inputs = ds['processed_input']
outputs = ds['processed_output']

In [8]:
import json
output_file = "/Users/frankli/Desktop/Yale Classes/Classes 2024/Ling/final/ling-final-project/llm-finetuning/data/lang8.jsonl"
with open(output_file, 'w', encoding='utf-8') as f:
    for input, output in zip(inputs, outputs):
        example = {"input": input, "output": output, "correction": []}
        f.write(json.dumps(example, ensure_ascii=False) + '\n')
    
    